# Create HDF5 DataFrame for S values from alphawave FEKO files
see: http://www.h5py.org/

+ This notebook was based on `RFcalcUC_v8.py` module
+ It is used for the **TE-007** spatial averaging working group calculations

## Set up

In [21]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn
sn.set()
%matplotlib inline

# Set current working directory to the data folder
%cd -0
%cd ../data

C:\Users\emeau\OneDrive\IT\Python\jupyter_notebooks\@Projects\spatial-averaging-wg\notebooks
C:\Users\emeau\OneDrive\IT\Python\jupyter_notebooks\@Projects\spatial-averaging-wg\data


## Functions

In [38]:
def h5store(filename, df, **kwargs):
    '''Save a DF with its metadata to a HDF5 file
       see: http://stackoverflow.com/questions/29129095/save-additional-attributes-in-pandas-dataframe
       INPUTS:
         filename = filename of the hdf5 file, including .hdf5 extension
               df = dataframe to be saved as hdf5 store
           kwargs = dict of metadata to store with the dataframe'''
    
    store = pd.HDFStore(filename)
    store.put('mydata', df)
    store.get_storer('mydata').attrs.metadata = kwargs
    store.close()

## Look at model metadata for the spatial averaging trials

In [39]:
# Read in "calc uc trials.xls" spreadsheet
trials = pd.read_excel('../data/spat avg trials.xlsx', 'trials', 
                       skiprows=1, index_col=0).fillna('')
# trials.drop("unit", axis=1, inplace=True)

trials

,unit,1,2
INPUT,,,
fMHz,MHz,900,
antenna,,IEC 62232 panel antenna,
Prad,W,80,
coordinate_origin,,centre of the middle feed segment,
xstart,m,0.5,
xend,m,14,
dx,m,0.1,
ystart,m,-7,
yend,m,7,


## Read in the E and H data from the FEKO zip files

In [40]:
# Set the trial number
trial = 1

# Get E and H FEKO data file locations
t = trials[str(trial)]
datadir = t['data_folder']
Efile = t['FEKO_Efile']
Hfile = t['FEKO_Hfile']

# Read in E-field FEKO data
print(f'{Efile = }')
Enames = ['x','y','z','Exr','Exi','Eyr','Eyi','Ezr','Ezi']
Edat = pd.read_csv(datadir + '/' + Efile, sep='\s+', skiprows=17, names=Enames)
Edat = Edat.sort_values(['x','y','z'])
display(Edat.head())

# Read in H-field FEKO data
print(f'{Hfile = }')
Hnames = ['x','y','z','Hxr','Hxi','Hyr','Hyi','Hzr','Hzi']
Hdat = pd.read_csv(datadir + '/' + Hfile, sep='\s+', skiprows=17, names=Hnames)
Hdat = Hdat.sort_values(['x','y','z'])
display(Hdat.head())

# Calculate SE, SH, Smax
SE = 0.5 * Edat.loc[:,'Exr':'Ezi'].pow(2).sum(axis=1) / 377
SH = 0.5 * Hdat.loc[:,'Hxr':'Hzi'].pow(2).sum(axis=1) * 377
Smax = np.max([SE,SH],axis=0)

S = pd.DataFrame(dict(x=Edat.x,y=Edat.y,z=Edat.z,SE=SE,SH=SH,Smax=Smax)).reset_index(drop=True)
S.head()

Efile = 'IEC-62232-panel-antenna-standard-mesh_Grid3DFE.efe.zip'


,x,y,z,Exr,Exi,Eyr,Eyi,Ezr,Ezi
0,0.5,-7.0,-3.0,0.002143,0.005977,0.348239,0.500341,-0.801133,-1.588188
19176,0.5,-7.0,-2.9,-0.005226,0.004985,-0.034064,0.622484,0.222759,-1.885471
38352,0.5,-7.0,-2.8,-0.008655,-0.001897,-0.386580,0.460931,1.224005,-1.515189
57528,0.5,-7.0,-2.7,-0.005113,-0.009175,-0.526968,0.137047,1.794559,-0.684281
76704,0.5,-7.0,-2.6,0.003041,-0.011303,-0.430148,-0.148068,1.809666,0.204564


Hfile = 'IEC-62232-panel-antenna-standard-mesh_Grid3DFE.hfe.zip'


,x,y,z,Hxr,Hxi,Hyr,Hyi,Hzr,Hzi
0,0.5,-7.0,-3.0,0.002329,0.004434,0.000096,0.000281,0.000050,0.000109
19176,0.5,-7.0,-2.9,-0.000593,0.005296,-0.000076,0.000306,-0.000036,0.000115
38352,0.5,-7.0,-2.8,-0.003431,0.004217,-0.000230,0.000227,-0.000097,0.000061
57528,0.5,-7.0,-2.7,-0.004992,0.001834,-0.000308,0.000084,-0.000102,-0.000014
76704,0.5,-7.0,-2.6,-0.004944,-0.000660,-0.000299,-0.000060,-0.000058,-0.000062


,x,y,z,SE,SH,Smax
0,0.5,-7.0,-3.0,0.004689,0.004748,0.004748
1,0.5,-7.0,-2.9,0.005296,0.005374,0.005374
2,0.5,-7.0,-2.8,0.005512,0.005594,0.005594
3,0.5,-7.0,-2.7,0.005286,0.005353,0.005353
4,0.5,-7.0,-2.6,0.004674,0.004708,0.004708


## Save the S DataFrame as hdf5 store with metadata

In [44]:
# Set the file name of the hdf5 store
Sfile = datadir + '.hdf5'

# Set the names of the stored S data
Snames = t.to_dict()
display(Snames)

{'fMHz': 900,
 'antenna': 'IEC 62232 panel antenna',
 'Prad': 80,
 'coordinate_origin': 'centre of the middle feed segment',
 'xstart': 0.5,
 'xend': 14,
 'dx': 0.1,
 'ystart': -7,
 'yend': 7,
 'dy': 0.1,
 'zstart': -3,
 'zend': 3,
 'dz': 0.1,
 'bearing': 'main beam along x axis',
 'data_folder': 'MBSpanel',
 'FEKO_Efile': 'IEC-62232-panel-antenna-standard-mesh_Grid3DFE.efe.zip',
 'FEKO_Hfile': 'IEC-62232-panel-antenna-standard-mesh_Grid3DFE.hfe.zip',
 'Notes': ''}

In [47]:
# Save the data into an hdf5 store
h5store(datadir + '/' + Sfile, S, **Snames)
print(f'S DataFrame saved to: {Sfile}')
print(f'len(S) = {len(S):,d}')

S DataFrame saved to: MBSpanel.hdf5
len(S) = 1,169,736
